In [1]:
# 라이브러리 임포트
import math
import os
import random

import numpy as np
import pandas as pd

import librosa
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# 하이퍼파라미터
args = {
    "train_path" : "/kaggle/input/2024-outta-basic-p-3/train/train",
    "test_path" : "/kaggle/input/2024-outta-basic-p-3/test/test",
    "submit_path" : "/kaggle/input/2024-outta-basic-p-3/sample_submission.csv",
    "extract_features" : "spectral",  # "rhythm"과 "spectral" 중에 선택하세요.
    "batch_size" : 64,
    "num_labels" : 2,
    "epochs" : 20,
    "lr" : 1e-4,
    "eps" : 1e-8,
    "seed_val" : 42     # 절대 수정하지 마세요.
}

In [3]:
# 랜덤시드 고정하기
seed = args["seed_val"]
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() :
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 디바이스 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
%pip install torchsummary
from torchsummary import summary as summary_## 모델 정보를 확인하기 위해 torchsummary 함수 import

## 모델의 형태를 출력하기 위한 함수
def summary_model(model, input_shape=(384,)):
    model = model.to(device)
    summary_(model, input_shape) ## (model, (input shape))

Note: you may need to restart the kernel to use updated packages.


In [5]:
# # (선택) Ipython 라이브러리를 이용해 학습에 사용될 음성을 직접 들어볼 수 있습니다.
# from IPython.display import Audio, display

# filename = '/kaggle/input/2024-outta-basic-p-3/train/train/blues/blues.00000.wav'  # 파일 주소
# y, sr = librosa.load(filename)
# audio_wdt = Audio(data=y,rate=sr)
# display(audio_wdt)

# **1. 데이터**

## **(1) Label Map**
{'강아지' : 0, '고양이' : 1} 등과 같은 형식으로, 머신러닝, 딥러닝 모델들은 feature나 label의 값들이 숫자(정수/실수)인 것만 처리할 수 있기 때문에, 문자열일 경우 숫자형으로 변환하여 처리합니다.

In [6]:
label_map = {'blues': 0,
             'classical': 1,
             'country': 2,
             'disco': 3,
             'hiphop': 4,
             'jazz': 5,
             'metal': 6,
             'pop': 7,
             'reggae': 8,
             'rock': 9}

## **(2) Feature Extract**
컴퓨터에 입력할 수 있도록 샘플링과 양자화를 거쳤다고 해도, 이를 바로 분류기에 넣는 것은 높은 성능을 보장할 수 없습니다.<br>
이는 음성 정보 안에 여러 주파수가 섞여 있으며 방대한 정보를 담고 있기 때문입니다.<br>
따라서 데이터 중 음성의 대표적인 성질을 나타낼 수 있는 handcrafted feature를 추출하여 사용하는 것이 필수적입니다.<br>
데이터에서 어떠한 특징을 어떠한 방식으로 추출하는지에 따라 분류기 성능에 큰 영향을 끼칠 수 있기 때문에 feature 추출을 위해서는 신중한 설계가 선행되어야 합니다.<br>

- 참고 1 : https://librosa.org/doc/latest/index.html
- 참고 2 : https://wikidocs.net/192879

In [7]:
def extract_rhythm_features(file_path):
    # 반환할 feature list 선언
    feature = []
    
    # (1-1) librosa.load(): extract_rhythm_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.
    y, sr = librosa.load(file_path)
    
    # (1-2) librosa.onset.onset_strength(): 앞서 load한 time-series를 함수의 입력으로 주어 onset_envelope를 추출합니다.
    onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
    
    # autocorrelation tempogram
    # (1-3) librosa.feature.tempogram(): 앞서 추출한 onset_envelope와 sr을 함수의 입력으로 주어 autocorrelation tempogram feature를 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    tempogram = librosa.feature.tempogram(onset_envelope=onset_envelope, sr=sr)
    
    # (1-4) autocorrelation tempogram에 대해 시간 축으로 평균을 내고 절대값을 취해 복소수를 제거함으로써 tempogram_feature를 얻습니다.
    tempogram_feature = np.abs(tempogram.mean(axis=1)) # (384, 1293) -> (384, ), 복소수를 없애기 위해 절대값 처리
    
    feature = tempogram_feature
    
    return feature

In [8]:
def extract_spectral_features(file_path):
    # 반환할 feature list 선언
    feature = []
    
    # (2-1) librosa.load(): extract_spectral_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.  
    y, sr = librosa.load(file_path)

    # (2-2) librosa.stft(): 앞서 load한 time-series를 함수의 입력으로 주어 stft를 추출하고, 절대값을 취해 복소수를 제거한 spectrogram을 추출합니다.  
    # * 이 때, 93ms의 물리적 간격으로 나뉘어 spectrogram이 생성될 수 있도록 하이퍼파라미터 n_fft를 조절합니다. 어떠한 값을 넣어야 하는지는 공식 문서를 참고하세요.
    n_fft = int(0.093*sr)
    spectrogram = np.abs(librosa.stft(y, n_fft=n_fft))
    
    # (2-3) 앞서 얻은 spectrogram에 제곱 연산을 취해 power_spectrogram을 얻습니다.  
    power_spectrogram = spectrogram ** 2
    
    # (2-4) librosa.feature.melspectrogram(): 앞서 얻은 power_spectrogram을 함수의 입력으로 주어 melspectrogram을 추출합니다.  
    melspectrogram = librosa.feature.melspectrogram(y=y,sr=sr,S=power_spectrogram, n_fft=n_fft)
    
    # (2-5) librosa.power_to_db(): 앞서 얻은 melspectrogram을 함수의 입력으로 주어 db scale로 변환된 melspectrogram_db를 추출합니다.  
    melspectrogram_db = librosa.power_to_db(melspectrogram)
    
    # chromagram
    # (2-6) librosa.feature.chroma_stft(): (2-3)에서 얻은 power_spectrogram을 함수의 입력으로 주어 chromagram을 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    chromagram = librosa.feature.chroma_stft(y=y,sr=sr,S=power_spectrogram)  # (12, 1293)
    
    # (2-7) chromagram에 대해 시간 축으로 평균을 내어 chromagram_feature를 얻습니다. 
    chromagram_feature = chromagram.mean(axis=1)  # (12, )
    
    # mfcc
    # (2-8) librosa.feature.mfcc(): (2-5)에서 얻은 melspectrogram_db를 함수의 입력으로 주어 mfcc를 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    mfcc = librosa.feature.mfcc(y=y,sr=sr,S=melspectrogram_db)  # (20, 1293)
    
    # (2-9) mfcc에 대해 시간 축으로 평균을 내어 mfcc_feature를 얻습니다.  
    mfcc_feature = mfcc.mean(axis=1)  # (20, )
    
    # (2-10) chromagram과 mfcc를 하나의 feature로 반환할 수 있도록 concatenate를 수행합니다.
    feature = np.concatenate((chromagram_feature, mfcc_feature))

    return feature

## **(3) Custom Dataset**
Custom Dataset 클래스는 크게 **__init__(), __len__(), 그리고 __getitem__()** 3개의 함수로 구현해야 합니다.

1.  __init__()
    - Dataset instance를 생성할 때 한번만 실행되는 함수로, 입력 이미지의 디렉토리와 라벨 정보 그리고 transform을 초기화 합니다.

2. __len__()
    - 데이터셋의 샘플 개수를 반환하는 함수 입니다.
    
3. __getitem__()
    - 주어진 인덱스에 해당하는 데이터 샘플을 데이터셋에서 불러오고 반환하는 함수 입니다.

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, split, extract_features, label_map):
        self.split = split.upper()
        self.root_path = root_path
        self.extract_features = extract_features
        self.label_map = label_map
        
        self.music = []
        self.label = []
        
        # (3-1) test 데이터 디렉토리 경로 초기화
        if self.split == 'TEST':
            musics = sorted([f for f in os.listdir(self.root_path) if f.endswith('.wav')])
            for music_ in musics:
                music_path = os.path.join(self.root_path, music_)
                self.music.append(music_path)
                
            
        # (3-2) train 데이터 디렉토리 경로, 라벨 초기화
        else:
            genres = sorted(os.listdir(self.root_path))
            for genre_ in genres:
                genre_path = os.path.join(self.root_path, genre_)
                if not os.path.isdir(genre_path):
                    continue
                musics = sorted([f for f in os.listdir(genre_path) if f.endswith('.wav')])
                for music_ in musics:
                    music_path = os.path.join(genre_path, music_)
                    self.music.append(music_path)
                    self.label.append(self.label_map[genre_])

    # 전체 데이터 샘플 개수 반환
    def __len__(self):
        return len(self.music)
    
    # 주어진 인덱스에 해당하는 데이터 반환
    def __getitem__(self, idx):        
        # (3-3) 음악 데이터에 feature extract 적용
        spectral_feature = extract_spectral_features(self.music[idx]).reshape(-1,1)
        rhythm_feature = extract_rhythm_features(self.music[idx]).reshape(-1,1)
        # spectral과 rhythm feature을 각각 minmax scale
        spectral_scaler = MinMaxScaler()
        rhythm_scaler = MinMaxScaler()
        
        normalized_spectral = spectral_scaler.fit_transform(spectral_feature)
        normalized_rhythm = rhythm_scaler.fit_transform(rhythm_feature).reshape(-1,1)
        # concatenate (384+32=416,)
        music_feature = np.concatenate((spectral_feature,rhythm_feature)).squeeze()
        
        # (3-4) test에 사용할 데이터 반환
        if self.split == 'TEST':
            return music_feature
        # (3-5) train에 사용할 데이터 반환
        else:
            label = self.label[idx]
            return music_feature, label

### **(4) 데이터셋과 데이터로더**

In [10]:
# (4-1) 훈련 및 테스트 데이터셋 로드
train_dataset = CustomDataset(root_path=args["train_path"], split='train', extract_features=args["extract_features"],label_map=label_map)
test_dataset = CustomDataset(root_path=args["test_path"], split='test', extract_features=args["extract_features"],label_map=label_map)
# (4-2) 데이터로더 정의
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=args["batch_size"], shuffle=True,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,)

In [11]:
print(f'Train Music Data\'s shape : {train_dataset.__getitem__(913)[0].shape}')   # train_dataset의 반환값 : music_feature, label
print(f'Test Music Data\'s shape : {test_dataset.__getitem__(299).shape}')        # test_dataset의 반환값 : music_feature

Train Music Data's shape : (416,)
Test Music Data's shape : (416,)


In [12]:
print(train_dataset.__len__())
print(test_dataset.__len__())

914
300


# **3. 모델**
모델을 직접 설계하여 프로젝트를 수행하세요.

직접 쌓기만 하신다면, 어떤 모델이든 사용 가능합니다.

아래는 베이스라인 모델입니다.

> ```
>         Layer (type)               Output Shape         Param #
> =================================================================
>             Conv1d-1              [-1, 64, 384]             256
>               ReLU-2              [-1, 64, 384]               0
>             Conv1d-3              [-1, 64, 384]          12,352
>               ReLU-4              [-1, 64, 384]               0
>          MaxPool1d-5              [-1, 64, 192]               0
>             Conv1d-6             [-1, 128, 192]          24,704
>               ReLU-7             [-1, 128, 192]               0
>             Conv1d-8             [-1, 128, 192]          49,280
>               ReLU-9             [-1, 128, 192]               0
>         MaxPool1d-10              [-1, 128, 96]               0
>            Conv1d-11              [-1, 256, 96]          98,560
>              ReLU-12              [-1, 256, 96]               0
>            Conv1d-13              [-1, 256, 96]         196,864
>              ReLU-14              [-1, 256, 96]               0
>            Conv1d-15              [-1, 256, 96]         196,864
>              ReLU-16              [-1, 256, 96]               0
>         MaxPool1d-17              [-1, 256, 48]               0
>            Conv1d-18              [-1, 512, 48]         393,728
>              ReLU-19              [-1, 512, 48]               0
>            Conv1d-20              [-1, 512, 48]         786,944
>              ReLU-21              [-1, 512, 48]               0
>            Conv1d-22              [-1, 512, 48]         786,944
>              ReLU-23              [-1, 512, 48]               0
>         MaxPool1d-24              [-1, 512, 24]               0
>            Conv1d-25              [-1, 512, 24]         786,944
>              ReLU-26              [-1, 512, 24]               0
>            Conv1d-27              [-1, 512, 24]         786,944
>              ReLU-28              [-1, 512, 24]               0
>            Conv1d-29              [-1, 512, 24]         786,944
>              ReLU-30              [-1, 512, 24]               0
>         MaxPool1d-31              [-1, 512, 12]               0
> AdaptiveAvgPool1d-32               [-1, 512, 1]               0
>           Flatten-33                  [-1, 512]               0
>            Linear-34                   [-1, 10]           5,130
> =================================================================
> Total params: 4,912,458
> Trainable params: 4,912,458
> Non-trainable params: 0
> ----------------------------------------------------------------
> Input size (MB): 0.00
> Forward/backward pass size (MB): 4.74
> Params size (MB): 18.74
> Estimated Total Size (MB): 23.48
> ----------------------------------------------------------------
> ```


In [13]:
# (5) 모델 설계
class Model(nn.Module):
    def __init__(self):
        """
        모델 초기화 함수입니다.
        """
        super(Model, self).__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1),  # Conv1d-1
            nn.BatchNorm1d(64),
            nn.ReLU(),                                                            # ReLU-2
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1), # Conv1d-3
            nn.BatchNorm1d(64),
            nn.ReLU(),                                                            # ReLU-4
            nn.MaxPool1d(kernel_size=2)                                           # MaxPool1d-5
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1), # Conv1d-6
            nn.BatchNorm1d(128),
            nn.ReLU(),                                                             # ReLU-7
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),# Conv1d-8
            nn.BatchNorm1d(128),
            nn.ReLU(),                                                             # ReLU-9
            nn.MaxPool1d(kernel_size=2)                                            # MaxPool1d-10
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),# Conv1d-11
            nn.BatchNorm1d(256),
            nn.ReLU(),                                                             # ReLU-12
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),# Conv1d-13
            nn.BatchNorm1d(256),
            nn.ReLU(),                                                             # ReLU-14
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),# Conv1d-15
            nn.BatchNorm1d(256),
            nn.ReLU(),                                                             # ReLU-16
            nn.MaxPool1d(kernel_size=2)                                            # MaxPool1d-17
        )
        
        self.conv_block4 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),# Conv1d-18
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-19
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),# Conv1d-20
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-21
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),# Conv1d-22
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-23
            nn.MaxPool1d(kernel_size=2)                                            # MaxPool1d-24
        )
        
        self.conv_block5 = nn.Sequential(
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),# Conv1d-25
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-26
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),# Conv1d-27
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-28
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),# Conv1d-29
            nn.BatchNorm1d(512),
            nn.ReLU(),                                                             # ReLU-30
            nn.MaxPool1d(kernel_size=2),                                        # MaxPool1d-31
            nn.AdaptiveAvgPool1d(1),  # AdaptiveAvgPool1d-32
            nn.Flatten()           # Flatten-33
        )
        # Classifier block
        self.classifier = nn.Sequential(
            nn.Linear(512, 10), # Linear-34, class=10
            nn.BatchNorm1d(10),
        )

    def forward(self, x):
        """
        모델의 순전파 함수입니다.

        Args:
            x (torch.Tensor): 입력 데이터

        Returns:
            torch.Tensor: 출력 예측값
        """
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = x.squeeze(1)
        x = self.classifier(x)
        
        return x

In [14]:
# 모델을 장치로 이동
model = Model().to(device)

# 가중치 초기화 함수 정의
def initialize_weights(m):
    if isinstance(m, nn.Conv1d):
        nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')  # He 초기화
        if m.bias is not None:
            nn.init.zeros_(m.bias)  # 바이어스를 0으로 초기화

# 모델에 초기화 함수 적용
model.apply(initialize_weights)

# 모델 요약 정보 출력
summary_model(model, input_shape=(1,416))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 64, 416]             256
       BatchNorm1d-2              [-1, 64, 416]             128
              ReLU-3              [-1, 64, 416]               0
            Conv1d-4              [-1, 64, 416]          12,352
       BatchNorm1d-5              [-1, 64, 416]             128
              ReLU-6              [-1, 64, 416]               0
         MaxPool1d-7              [-1, 64, 208]               0
            Conv1d-8             [-1, 128, 208]          24,704
       BatchNorm1d-9             [-1, 128, 208]             256
             ReLU-10             [-1, 128, 208]               0
           Conv1d-11             [-1, 128, 208]          49,280
      BatchNorm1d-12             [-1, 128, 208]             256
             ReLU-13             [-1, 128, 208]               0
        MaxPool1d-14             [-1, 1

# **4. 학습**

In [15]:
def train(train_dataloader, model, device, args):
    """
    모델을 훈련시키는 함수입니다.

    Args:
        train_dataloader (DataLoader): 훈련 데이터가 포함된 데이터로더
        model (nn.Module): 훈련할 신경망 모델
        device (torch.device): 모델과 데이터를 올릴 장치 (CPU 또는 GPU)
        args (dict): 훈련 설정을 포함한 딕셔너리 (예: 에포크 수)

    Returns:
        None
    """
    # (6-1) 옵티마이저와 손실 함수 정의
    optimizer = torch.optim.Adam(model.parameters(),lr=args["lr"],eps=args["eps"])
    loss_fn = nn.CrossEntropyLoss()
    
    # 모델의 모든 파라미터의 기울기를 0으로 초기화
    model.zero_grad()
    
    # 지정된 에포크 수만큼 훈련 반복
    for epoch in range(args["epochs"]):
        print(f'Epoch {epoch + 1}/{args["epochs"]}')
        
        train_loss = 0
        train_acc = 0
        
        # 훈련 데이터로더에서 데이터를 반복적으로 가져옴
        for data, label in tqdm(train_dataloader):
            # 데이터를 부동 소수점 형식으로 변환하고 장치에 올림
            data = data.unsqueeze(1).float().to(device)
            label = label.long().to(device)
            # (6-2) 모델을 사용하여 예측 수행\
            pred = model(data)
            # (6-3) 예측값과 실제 라벨을 사용하여 손실 계산
            loss = loss_fn(pred,label)
            # (6-4) 옵티마이저의 기울기를 초기화
            optimizer.zero_grad()
            
            # (6-5) 역전파를 통해 기울기 계산
            loss.backward()
            
            # (6-6) 옵티마이저를 사용하여 모델의 파라미터 업데이트
            optimizer.step()
            
            # 총 손실 계산
            train_loss += loss.item()
            # 예측값에서 가장 높은 값의 인덱스를 선택
            pred = pred.argmax(dim=1)
            # 정확도 계산
            train_acc += (pred == label).sum().item()
            
        # 평균 손실과 정확도 계산
        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader.dataset)
            
        # 모델과 옵티마이저 상태를 저장
        os.makedirs("results", exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model': model,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
            }, f'./results/model_state_dict_epoch_{epoch+1}.pth')
            
        # 현재 에포크의 손실과 정확도 출력
        print(f'CheckPoint : model_state_dict_epoch_{epoch+1}.pth')
        print(f'Train Loss : {train_loss}, Train Accuracy : {train_acc}\n')
        

if __name__ == "__main__":
    # 훈련 함수 호출
    train(train_dataloader, model, device, args)

Epoch 1/20


100%|██████████| 15/15 [04:32<00:00, 18.17s/it]


CheckPoint : model_state_dict_epoch_1.pth
Train Loss : 1.7486816724141439, Train Accuracy : 0.40043763676148797

Epoch 2/20


100%|██████████| 15/15 [04:27<00:00, 17.83s/it]


CheckPoint : model_state_dict_epoch_2.pth
Train Loss : 1.341270112991333, Train Accuracy : 0.587527352297593

Epoch 3/20


...

CheckPoint : model_state_dict_epoch_18.pth
Train Loss : 0.45580325921376547, Train Accuracy : 0.9923413566739606

Epoch 19/20


100%|██████████| 15/15 [04:22<00:00, 17.47s/it]


CheckPoint : model_state_dict_epoch_19.pth
Train Loss : 0.43005738655726117, Train Accuracy : 0.9967177242888403

Epoch 20/20


100%|██████████| 15/15 [04:19<00:00, 17.28s/it]

CheckPoint : model_state_dict_epoch_20.pth
Train Loss : 0.41964129408200584, Train Accuracy : 0.9956236323851203



# **5. 평가**

In [16]:
def test(test_dataloader, model, device):
    """
    모델의 예측을 수행하는 함수입니다.

    Args:
        test_dataloader (DataLoader): 테스트 데이터가 포함된 데이터로더
        model (nn.Module): 예측에 사용할 신경망 모델
        device (torch.device): 모델과 데이터를 올릴 장치 (CPU 또는 GPU)

    Returns:
        preds (list): 예측된 클래스의 정수 인덱스 리스트
    """
    # 모델을 평가 모드로 전환
    model.eval()
    # 예측 결과를 저장할 리스트 초기화
    preds = []
    
    # 데이터로더에서 데이터를 반복적으로 가져옴
    for data in tqdm(test_dataloader):
        # 데이터를 부동 소수점 형식으로 변환하고 장치에 올림
        data = data.unsqueeze(1).float().to(device)
        # 기울기 계산을 비활성화하여 예측 수행
        with torch.no_grad():
            # (7-1) 모델을 사용하여 예측 수행
            pred = model(data)
            # (7-2) argmax를 이용하여 예측 결과에서 가장 높은 값의 인덱스를 선택
            pred = pred.argmax(dim=1)
            
            # (7-3) 예측 결과를 리스트에 추가
            preds.append(pred)
            
    return preds
            

def int_to_label(label_map, predictions_int):
    """
    정수 형태의 예측값 리스트를 클래스 라벨로 변환하는 함수입니다.

    Args:
        label_map (dict): 클래스 라벨과 정수 인덱스 매핑이 저장된 딕셔너리
        predictions_int (list): 정수 형태의 예측값 리스트

    Returns:
        predicted_labels (list): 문자열 형태의 예측 클래스 라벨 리스트
    """
    # 정수 예측값을 클래스 라벨로 변환하여 리스트에 저장
    predicted_labels = [list(label_map.keys())[list(label_map.values()).index(pred)] for pred in predictions_int]
    
    return predicted_labels


if __name__ == "__main__":
    # 예측값을 얻기 위해 test 함수 호출
    preds = test(test_dataloader, model, device)
    
    # 정수 예측값을 클래스 라벨로 변환
    preds = int_to_label(label_map, preds)


100%|██████████| 300/300 [01:30<00:00,  3.33it/s]


In [17]:
submit = pd.read_csv(args["submit_path"])
submit['genre'] = preds
submit.to_csv('submission_p3.csv', index=False)

In [18]:
submit

,music,genre
0,0001.wav,jazz
1,0002.wav,disco
2,0003.wav,jazz
3,0004.wav,rock
4,0005.wav,hiphop
...,...,...
295,0296.wav,classical
296,0297.wav,metal
297,0298.wav,disco
298,0299.wav,rock
